## Open Dart 개발 가이드 - 상장기업 재무정보

### 1. 단일회사 주요계정
> __보고서 위치__
* 없음

> __기능__
* 정기보고서 내에 XBRL재무제표의 주요계정과목을 제공

> __획득 가능 주요 정보__
* account_nm : 계정명(자본총계/유동자산 등)
* fs_div : 개별/연결구분(CFS:연결/OFS:개별)
* sj_div : 재무제표구분(BS/IS)
* thstrm_dt : 당기일자(2018.09.30 현재)
* thstrm_amount : 당기금액
* frmtrm_dt : 전기일자(2017.01.01 ~ 2017.12.31)
* frmtrm_amount : 전기금액
* bfefrmtrm_dt : 전전기일자
* bfefrmtrm_amount : 전전기금액(사업보고서의 경우에만 출력)

In [31]:
import requests
import json
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from io import BytesIO
import zipfile
from bs4 import BeautifulSoup
import os
import sys

In [3]:
# 요청인자
crtfc_key = '27d86452b033fe973df6f1c980c37a2586801a1a'
corp_code = '00293886'  #LG전자
bsns_year = '2018'
reprt_code = '11011'

In [4]:
url_basic = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"
url_rest = "?crtfc_key={}&corp_code={}&bsns_year={}&reprt_code={}".format(crtfc_key, corp_code, bsns_year, reprt_code)
url = url_basic + url_rest
resp = requests.get(url)
webpage = resp.content.decode('UTF-8')
webpage

'{"status":"000","message":"정상","list":[{"rcept_no":"20190820000266","reprt_code":"11011","bsns_year":"2018","corp_code":"00293886","stock_code":"044340","fs_div":"CFS","fs_nm":"연결재무제표","sj_div":"BS","sj_nm":"재무상태표","account_nm":"유동자산","thstrm_nm":"제 33 기","thstrm_dt":"2018.12.31 현재","thstrm_amount":"106,479,426,061","frmtrm_nm":"제 32 기","frmtrm_dt":"2017.12.31 현재","frmtrm_amount":"90,207,795,983","bfefrmtrm_nm":"제 31 기","bfefrmtrm_dt":"2016.12.31 현재","bfefrmtrm_amount":"95,924,754,923","ord":"1"},{"rcept_no":"20190820000266","reprt_code":"11011","bsns_year":"2018","corp_code":"00293886","stock_code":"044340","fs_div":"CFS","fs_nm":"연결재무제표","sj_div":"BS","sj_nm":"재무상태표","account_nm":"비유동자산","thstrm_nm":"제 33 기","thstrm_dt":"2018.12.31 현재","thstrm_amount":"114,045,036,073","frmtrm_nm":"제 32 기","frmtrm_dt":"2017.12.31 현재","frmtrm_amount":"110,797,774,844","bfefrmtrm_nm":"제 31 기","bfefrmtrm_dt":"2016.12.31 현재","bfefrmtrm_amount":"109,410,793,889","ord":"3"},{"rcept_no":"20190820000266","r

In [5]:
account_nm_ls = []
fs_div_ls = []
sj_div_ls = []
frst_acqs_amount_ls = []
thstrm_dt_ls = []
thstrm_amount_ls = []
frmtrm_dt_ls = []
frmtrm_amount_ls = []
bfefrmtrm_dt_ls = []
bfefrmtrm_amount_ls = []
ord_ls = []

dict = json.loads(webpage)
for val in dict:
    if val == 'list':
        for ele in dict[val]:
            for sub_ele in ele:
                if sub_ele == 'ord': ord_ls.append(ele[sub_ele])
                elif sub_ele == 'account_nm': account_nm_ls.append(ele[sub_ele])
                elif sub_ele == 'fs_div': fs_div_ls.append(ele[sub_ele])
                elif sub_ele == 'sj_div': sj_div_ls.append(ele[sub_ele])
                elif sub_ele == 'frst_acqs_amount': frst_acqs_amount_ls.append(ele[sub_ele])
                elif sub_ele == 'thstrm_dt': thstrm_dt_ls.append(ele[sub_ele])
                elif sub_ele == 'thstrm_amount': thstrm_amount_ls.append(ele[sub_ele])
                elif sub_ele == 'frmtrm_dt': frmtrm_dt_ls.append(ele[sub_ele])
                elif sub_ele == 'frmtrm_amount': frmtrm_amount_ls.append(ele[sub_ele])
                elif sub_ele == 'bfefrmtrm_dt': bfefrmtrm_dt_ls.append(ele[sub_ele])
                elif sub_ele == 'bfefrmtrm_amount': bfefrmtrm_amount_ls.append(ele[sub_ele])
                
len(account_nm_ls)

26

In [6]:
data = {
    '정렬순서': ord_ls,
    '계정명': account_nm_ls,
    '연결/개별': fs_div_ls,
    'FS구분': sj_div_ls,
    thstrm_dt_ls[0]: thstrm_amount_ls,
    frmtrm_dt_ls[0]: frmtrm_amount_ls,
    bfefrmtrm_dt_ls[0]: bfefrmtrm_amount_ls
}

df_fs_abbr = DataFrame(data)
df_fs_abbr = df_fs_abbr.sort_values(['연결/개별','FS구분'], ascending=True)
df_fs_abbr

,정렬순서,계정명,연결/개별,FS구분,2018.12.31 현재,2017.12.31 현재,2016.12.31 현재
0,1,유동자산,CFS,BS,"106,479,426,061","90,207,795,983","95,924,754,923"
1,3,비유동자산,CFS,BS,"114,045,036,073","110,797,774,844","109,410,793,889"
2,5,자산총계,CFS,BS,"220,524,462,134","201,005,570,827","205,335,548,812"
3,7,유동부채,CFS,BS,"85,709,220,221","87,533,713,235","108,407,185,346"
4,9,비유동부채,CFS,BS,"16,108,623,326","15,954,658,431","5,307,270,090"
5,11,부채총계,CFS,BS,"101,817,843,547","103,488,371,666","113,714,455,436"
6,13,자본금,CFS,BS,"8,936,712,500","8,179,320,500","8,179,320,500"
7,17,이익잉여금,CFS,BS,"60,774,263,265","49,082,429,362","38,781,006,850"
8,21,자본총계,CFS,BS,"118,706,618,587","97,517,199,161","91,621,093,376"
9,23,매출액,CFS,IS,"330,556,897,475","260,728,203,252","213,152,599,991"


### 2. 다중회사 주요계정
> __보고서 위치__
* 없음

> __기능__
* 여러회사의 주요계정과목을 동시 비교

> __획득 가능 주요 정보__
* account_nm : 계정명(자본총계/유동자산 등)
* fs_div : 개별/연결구분(CFS:연결/OFS:개별)
* sj_div : 재무제표구분(BS/IS)
* thstrm_dt : 당기일자(2018.09.30 현재)
* thstrm_amount : 당기금액
* frmtrm_dt : 전기일자(2017.01.01 ~ 2017.12.31)
* frmtrm_amount : 전기금액
* bfefrmtrm_dt : 전전기일자
* bfefrmtrm_amount : 전전기금액(사업보고서의 경우에만 출력)

In [7]:
# 요청인자
corp_code = '00293886,00126380'  #LG전자,삼성전자

In [8]:
url_basic = "https://opendart.fss.or.kr/api/fnlttMultiAcnt.json"
url_rest = "?crtfc_key={}&corp_code={}&bsns_year={}&reprt_code={}".format(crtfc_key, corp_code, bsns_year, reprt_code)
url = url_basic + url_rest
resp = requests.get(url)
webpage = resp.content.decode('UTF-8')
webpage

'{"status":"000","message":"정상","list":[{"rcept_no":"20190401004781","reprt_code":"11011","bsns_year":"2018","corp_code":"00126380","stock_code":"005930","fs_div":"CFS","fs_nm":"연결재무제표","sj_div":"BS","sj_nm":"재무상태표","account_nm":"유동자산","thstrm_nm":"제 50 기","thstrm_dt":"2018.12.31 현재","thstrm_amount":"174,697,424,000,000","frmtrm_nm":"제 49 기","frmtrm_dt":"2017.12.31 현재","frmtrm_amount":"146,982,464,000,000","bfefrmtrm_nm":"제 48 기","bfefrmtrm_dt":"2016.12.31 현재","bfefrmtrm_amount":"141,429,704,000,000","ord":"1"},{"rcept_no":"20190401004781","reprt_code":"11011","bsns_year":"2018","corp_code":"00126380","stock_code":"005930","fs_div":"CFS","fs_nm":"연결재무제표","sj_div":"BS","sj_nm":"재무상태표","account_nm":"비유동자산","thstrm_nm":"제 50 기","thstrm_dt":"2018.12.31 현재","thstrm_amount":"164,659,820,000,000","frmtrm_nm":"제 49 기","frmtrm_dt":"2017.12.31 현재","frmtrm_amount":"154,769,626,000,000","bfefrmtrm_nm":"제 48 기","bfefrmtrm_dt":"2016.12.31 현재","bfefrmtrm_amount":"120,744,620,000,000","ord":"3"},{"rce

In [9]:
stock_code_ls = []
account_nm_ls = []
fs_div_ls = []
sj_div_ls = []
frst_acqs_amount_ls = []
thstrm_dt_ls = []
thstrm_amount_ls = []
frmtrm_dt_ls = []
frmtrm_amount_ls = []
bfefrmtrm_dt_ls = []
bfefrmtrm_amount_ls = []
ord_ls = []

dict = json.loads(webpage)
for val in dict:
    if val == 'list':
        for ele in dict[val]:
            for sub_ele in ele:
                if sub_ele == 'ord': ord_ls.append(ele[sub_ele])
                elif sub_ele == 'stock_code': stock_code_ls.append(ele[sub_ele])
                elif sub_ele == 'account_nm': account_nm_ls.append(ele[sub_ele])
                elif sub_ele == 'fs_div': fs_div_ls.append(ele[sub_ele])
                elif sub_ele == 'sj_div': sj_div_ls.append(ele[sub_ele])
                elif sub_ele == 'frst_acqs_amount': frst_acqs_amount_ls.append(ele[sub_ele])
                elif sub_ele == 'thstrm_dt': thstrm_dt_ls.append(ele[sub_ele])
                elif sub_ele == 'thstrm_amount': thstrm_amount_ls.append(ele[sub_ele])
                elif sub_ele == 'frmtrm_dt': frmtrm_dt_ls.append(ele[sub_ele])
                elif sub_ele == 'frmtrm_amount': frmtrm_amount_ls.append(ele[sub_ele])
                elif sub_ele == 'bfefrmtrm_dt': bfefrmtrm_dt_ls.append(ele[sub_ele])
                elif sub_ele == 'bfefrmtrm_amount': bfefrmtrm_amount_ls.append(ele[sub_ele])
                
len(account_nm_ls)

52

In [10]:
data = {
    '정렬순서': ord_ls,
    '주식코드': stock_code_ls,
    '계정명': account_nm_ls,
    '연결/개별': fs_div_ls,
    'FS구분': sj_div_ls,
    thstrm_dt_ls[0]: thstrm_amount_ls,
    frmtrm_dt_ls[0]: frmtrm_amount_ls,
    bfefrmtrm_dt_ls[0]: bfefrmtrm_amount_ls
}

df_fs_abbr = DataFrame(data)
df_fs_abbr = df_fs_abbr.sort_values(['연결/개별','FS구분'], ascending=True)
df_fs_abbr

,정렬순서,주식코드,계정명,연결/개별,FS구분,2018.12.31 현재,2017.12.31 현재,2016.12.31 현재
0,1,005930,유동자산,CFS,BS,"174,697,424,000,000","146,982,464,000,000","141,429,704,000,000"
1,3,005930,비유동자산,CFS,BS,"164,659,820,000,000","154,769,626,000,000","120,744,620,000,000"
2,5,005930,자산총계,CFS,BS,"339,357,244,000,000","301,752,090,000,000","262,174,324,000,000"
3,7,005930,유동부채,CFS,BS,"69,081,510,000,000","67,175,114,000,000","54,704,095,000,000"
4,9,005930,비유동부채,CFS,BS,"22,522,557,000,000","20,085,548,000,000","14,507,196,000,000"
5,11,005930,부채총계,CFS,BS,"91,604,067,000,000","87,260,662,000,000","69,211,291,000,000"
6,13,005930,자본금,CFS,BS,"897,514,000,000","897,514,000,000","897,514,000,000"
7,17,005930,이익잉여금,CFS,BS,"242,698,956,000,000","215,811,200,000,000","193,086,317,000,000"
8,21,005930,자본총계,CFS,BS,"247,753,177,000,000","214,491,428,000,000","192,963,033,000,000"
26,1,044340,유동자산,CFS,BS,"106,479,426,061","90,207,795,983","95,924,754,923"


### 3. 재무제표 원본파일(XBRL)
> __보고서 위치__
* 없음

> __기능__
* 재무제표 원본파일 제공

> __획득 가능 주요 정보__
* account_nm : 계정명(자본총계/유동자산 등)
* fs_div : 개별/연결구분(CFS:연결/OFS:개별)
* sj_div : 재무제표구분(BS/IS)
* thstrm_dt : 당기일자(2018.09.30 현재)
* thstrm_amount : 당기금액
* frmtrm_dt : 전기일자(2017.01.01 ~ 2017.12.31)
* frmtrm_amount : 전기금액
* bfefrmtrm_dt : 전전기일자
* bfefrmtrm_amount : 전전기금액(사업보고서의 경우에만 출력)

In [11]:
# 요청인자
crtfc_key = '27d86452b033fe973df6f1c980c37a2586801a1a'
rcept_no = '20190401004781' # 삼성전자 2018년 사업보고서, OpenDart_DevGuide_공시정보 > 1. 공시정보 > 공시검색
reprt_code = '11011'

In [12]:
url_basic = "https://opendart.fss.or.kr/api/fnlttXbrl.xml"
url_rest = "?crtfc_key={}&rcept_no={}&reprt_code={}".format(crtfc_key, rcept_no, reprt_code)
url = url_basic + url_rest
resp = requests.get(url)
data = BytesIO(resp.content)
my_zip = zipfile.ZipFile(data)
my_zip.namelist()

['00126380_2011-04-30.xbrl',
 '00126380_entry_point_2011-04-30.xsd',
 'ifrs_for_00126380/dimensions_ifrs_for_00126380/dim_ifrs_for_00126380_2011-04-30.xml',
 'ifrs_for_00126380/dimensions_ifrs_for_00126380/cal_ifrs_for_00126380_2011-04-30.xml',
 'ifrs_for_00126380/dimensions_ifrs_for_00126380/pre_ifrs_for_00126380_2011-04-30.xml',
 'ifrs_for_00126380/dimensions_ifrs_for_00126380/role_ifrs_for_00126380-dim_2011-04-30.xsd',
 'labels/lab_00126380-ko_2011-04-30.xml',
 'labels/lab_00126380-en_2011-04-30.xml']

In [56]:
data = my_zip.read(my_zip.namelist()[0])
soup = BeautifulSoup(data, "lxml")
tag_list = soup.find_all('dart:bondsissued', decimals="-6") #'dart-gcd:DomesticExchange'
tag_list
# for tag in tag_list:
#     if tag.name == 'dart-gcd:authorname':
#         print(tag.text)

[<dart:bondsissued contextref="BPFY2016eFY_ifrs_ConsolidatedAndSeparateFinancialStatementsAxis_ifrs_ConsolidatedMember" decimals="-6" unitref="KRW">58542000000</dart:bondsissued>,
 <dart:bondsissued contextref="BPFY2016eFY_ifrs_ConsolidatedAndSeparateFinancialStatementsAxis_ifrs_SeparateMember" decimals="-6" unitref="KRW">58542000000</dart:bondsissued>,
 <dart:bondsissued contextref="PFY2017eFY_ifrs_ConsolidatedAndSeparateFinancialStatementsAxis_ifrs_ConsolidatedMember" decimals="-6" unitref="KRW">953361000000</dart:bondsissued>,
 <dart:bondsissued contextref="PFY2017eFY_ifrs_ConsolidatedAndSeparateFinancialStatementsAxis_ifrs_SeparateMember" decimals="-6" unitref="KRW">46808000000</dart:bondsissued>,
 <dart:bondsissued contextref="CFY2018eFY_ifrs_ConsolidatedAndSeparateFinancialStatementsAxis_ifrs_ConsolidatedMember" decimals="-6" unitref="KRW">961972000000</dart:bondsissued>,
 <dart:bondsissued contextref="CFY2018eFY_ifrs_ConsolidatedAndSeparateFinancialStatementsAxis_ifrs_SeparateMe